In [1]:
import pandas as pd
import os
import pickle
import time as t
from pathlib import Path
import datetime as dt
from datetime import datetime
from datetime import timedelta
import pandas.io.parsers
import warnings
warnings.filterwarnings('ignore')

In [2]:
instrument = ("NIFTY").upper()

start_date = datetime.strptime("2023-01-01", "%Y-%m-%d").date()
end_date = datetime.strptime("2023-02-28", "%Y-%m-%d").date()
data_root_path = Path(r"C:\Data\Dataset\Dict")
output_csv_path = Path(r"C:\My_Code\Raghav")
param_file_path = Path(r"C:\Users\User\Desktop\Raghav_Param (1).csv")

param_data = pd.read_csv(param_file_path)
display(param_data)

,Entry_Time,Exit_Time,Entry_Weekday,StopLoss,Max_Entries
0,09:20:59,15:20:59,Friday,20,10
1,09:30:59,15:20:59,Friday,20,10
2,09:40:59,15:20:59,Friday,20,10
3,09:50:59,15:20:59,Friday,20,10
4,10:00:59,15:20:59,Friday,20,10
...,...,...,...,...,...
940,14:30:59,15:20:59,Thursday,100,10
941,14:45:59,15:20:59,Thursday,100,10
942,15:00:59,15:20:59,Thursday,100,10
943,15:15:59,15:20:59,Thursday,100,10


In [3]:
# Pre Defined Functions

# Roundoff : This Funtion is Using A Value Which your Want to Convert in Whole Number of its given input Value (50, 100)
def Roundoff(x, base=0.05):
    return base * round(x / base)

# Filename :  This Function Creates a Filename based on your Date
""" In Our System -  For Pickle Dataframe : File Name format is (BN_01012021.pkl / NIFTY_01012021.pkl), it contains Day wise Data
                    For Pickle Dictionary : File Name Format is (BN_2021.pkl / NIFTY_2021.pkl), it contains yearly Data"""
def Filename(year, Instrument):
    year = year
    if Instrument == "NIFTY":
        filename = f"NIFTY_{year}.pkl"
    else:
        filename = f"{Instrument}_{year}.pkl"
    return filename, year

In [4]:
dates_range = [str(start_date+timedelta(days=x)) for x in range((end_date-start_date).days + 1)]
year_data = str(start_date.year)

In [5]:
column_list = ['Instrument', 'User_Entry_Time', 'User_Exit_Time', 'user_weekday', 'Stop_Loss_%', 'Max_Entries', 'Expiry','Entry_Date_1', 'Exit_Date_1', 'Entry_Weekday_1', 'Exit_Weekday_1', 'Entry_Time_1' ,'Exit_Time_1', 'Exit_Straddle_1', 'Stop_Loss_1', 'SL_Type_1' ,'CE_PNL_1', 'PE_PNL_1', 'Total_PNL_1']
market_opening_time = '09:15:59'
market_closing_time = '15:29:59'

In [6]:
def Entry_Function(df, expiry, entry_time, user_weekday, stop_loss_p):
    weekdays_name_list = ['Friday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday']
    dates_keys_list = sorted(list(df[expiry].keys()))
    weekday_list = list(map(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%A'), dates_keys_list))
    if user_weekday in weekday_list:
        try:
            start_index = weekday_list.index(user_weekday)
        except:
            try:
                weekday_index = weekdays_name_list.index(user_weekday)
                weekday_next_day = weekdays_name_list[weekday_index + 1]
                start_index = weekday_list.index(weekday_next_day)
            except:
                weekday_index = weekdays_name_list.index(user_weekday)
                weekday_next_day = weekdays_name_list[weekday_index + 2]
                start_index = weekday_list.index(weekday_next_day)
        date_key_list = dates_keys_list[start_index : ]
        entry_date = date_key_list[0]
        entry_weekday = (pd.Timestamp(entry_date)).day_name()

        if entry_time in (df[expiry][entry_date].keys()):
            ce_strk_set = list((df[expiry][entry_date][entry_time]["CE"].keys()))
            pe_strk_set = list((df[expiry][entry_date][entry_time]["PE"].keys()))
            common_strike = list(set(ce_strk_set) & set(pe_strk_set))
            price_diff, strikes_list = [], []
            for strk in common_strike:
                price_diff.append(abs((df[expiry][entry_date][entry_time]["CE"][strk]['close'])-(df[expiry][entry_date][entry_time]["PE"][strk]['close'])))
                strikes_list.append(strk)
                Common_Strike_Price = strk
                ATM_strike = strikes_list[price_diff.index(min(price_diff))]
                ep_ce = df[expiry][entry_date][entry_time]["CE"][ATM_strike]['close']
                ep_pe = df[expiry][entry_date][entry_time]["PE"][ATM_strike]['close']
                ep_straddle = ep_ce + ep_pe
                SL = ep_straddle + (ep_straddle * stop_loss_p)/100
        else:
            ATM_strike, ep_ce, ep_pe, ep_straddle, SL = "", "", "", "", "" 
    else:
        ATM_strike, ep_ce, ep_pe, ep_straddle, SL, date_key_list, entry_date, entry_weekday = "", "", "", "", "", "", "", ""
    return ATM_strike, ep_ce, ep_pe, ep_straddle, SL, date_key_list, entry_date, entry_weekday

In [7]:
def Rolling_Straddle_Positional(df, expiry_list, entry_time, exit_time, stop_loss_p, user_weekday, max_entries):    
    start = t.perf_counter()
    mergefile, len_of_column  = [], []
    user_entry_time, user_exit_time = entry_time, exit_time
    for expiry in expiry_list:
        sl_type = ""
        same_entry_list = []
        for k in range(1, max_entries):
            if (sl_type == "Hit") or (sl_type == ""):
                if (sl_type == "Hit"):
                    entry_time = exit_time
                    entry_weekday = (pd.Timestamp(exit_date)).day_name()
                    ATM_strike, ep_ce, ep_pe, ep_straddle, SL, date_key_list, entry_date, entry_weekday = Entry_Function(df, expiry, entry_time, entry_weekday, stop_loss_p)
                else:
                    entry_time = user_entry_time
                    ATM_strike, ep_ce, ep_pe, ep_straddle, SL, date_key_list, entry_date, entry_weekday = Entry_Function(df, expiry, entry_time, user_weekday, stop_loss_p)
            else:
                entry_time = user_entry_time
                entry_date = first_entry_date
                ATM_strike, ep_ce, ep_pe, ep_straddle, SL, date_key_list, entry_date, entry_weekday = Entry_Function(df, expiry, entry_time, user_weekday, stop_loss_p)

            for d in date_key_list:
                if (d == entry_date) & ((pd.Timestamp(entry_date)).day_name() == "Thursday"):
                    time_list = sorted([key for key, value in df[expiry][d].items() if entry_time < key <= exit_time])
                elif d == entry_date:
                    time_list = sorted([key for key, value in df[expiry][d].items() if entry_time < key <= market_closing_time])             
                elif d == date_key_list[-1]:
                    time_list = sorted([key for key, value in df[expiry][d].items() if market_opening_time < key <= exit_time])
                else:
                    time_list = sorted([key for key, value in df[expiry][d].items() if market_opening_time < key <= market_closing_time])

                for time in time_list:
                    try:
                        straddle = df[expiry][d][time]["CE"][ATM_strike]['close'] + df[expiry][d][time]["PE"][ATM_strike]['close']
                        if(SL <= straddle):  
                            sl_type = "Hit"
                            exit_time = time
                            exit_straddle = straddle
                            CE_PNL = ep_ce - df[expiry][d][time]["CE"][ATM_strike]['close']
                            PE_PNL = ep_pe - df[expiry][d][time]["PE"][ATM_strike]['close']
                            exit_date = d
                            exit_weekday = (pd.Timestamp(exit_date)).day_name()
                            ATM_strike = ATM_strike
                            ep_ce = ep_ce
                            ep_pe = ep_pe
                            ep_straddle = ep_straddle
                            break
                        else:
                            sl_type = "Not Hit"
                            blank = 0
                            exit_straddle = straddle
                            exit_time = time
                            CE_PNL = ep_ce - df[expiry][d][time]["CE"][ATM_strike]['close']
                            PE_PNL = ep_pe - df[expiry][d][time]["PE"][ATM_strike]['close'] 
                            exit_date = d
                            exit_weekday = (pd.Timestamp(exit_date)).day_name()
                            ATM_strike = ATM_strike
                            ep_ce = ep_ce
                            ep_pe = ep_pe
                            ep_straddle = ep_straddle
                    except:
                        sl_type, exit_time, CE_PNL, PE_PNL, exit_date, exit_weekday = "", "", "", "", "", ""
                        pass

                if sl_type == "Hit":
                    output_df = pd.DataFrame(columns= [f'ATM_strike_{k}', f'Entry_Price_CE_{k}', f'Entry_Price_PE_{k}',f'Entry_Straddle_{k}',f'Entry_Date_{k}', f'Entry_Weekday_{k}', f'Entry_Time_{k}', f'Stop_Loss_{k}', f'SL_Type_{k}',f'Exit_Date_{k}', f'Exit_Weekday_{k}', f'Exit_Time_{k}', f'Exit_Straddle_{k}', f'CE_PNL_{k}', f'PE_PNL_{k}', f'Total_PNL_{k}'])
                    c = 0
                    output_df.loc[c] = [ATM_strike, ep_ce, ep_pe, ep_straddle, entry_date, entry_weekday, entry_time, SL, sl_type,exit_date, exit_weekday, exit_time, exit_straddle, CE_PNL, PE_PNL, CE_PNL+PE_PNL]  
                    same_entry_list.append(output_df)
                    break
                else:
                    pass

            if sl_type == "Not Hit":
                n = k
                break
            else:
                n = k + 1
                pass
        len_of_column.append(n)
        
        
        if len(same_entry_list) > 0:
            df_merged = pd.concat(same_entry_list, axis=1)
            df_merged['User_Entry_Time'] =  user_entry_time 
            df_merged['User_Exit_Time'] = user_exit_time 
            df_merged['user_weekday'] = user_weekday 
            df_merged['Stop_Loss_%'] = stop_loss_p
            df_merged['Max_Entries'] = max_entries
            df_merged['Expiry'] = expiry
            df_merged['Instrument'] = instrument
            x_y_columns_list = sorted(list(set(column_list) ^ set(df_merged.columns)))
            new_order = column_list + x_y_columns_list
            df_merged = df_merged.reindex(columns = sorted(new_order))
            output_df = pd.DataFrame(columns= [f'ATM_strike_{n}', f'Entry_Price_CE_{n}', f'Entry_Price_PE_{n}', f'Entry_Straddle_{n}', f'Entry_Date_{n}', f'Entry_Weekday_{n}',  f'Entry_Time_{n}', f'Exit_Date_{n}', f'Exit_Weekday_{n}', f'Exit_Time_{n}', f'Exit_Straddle_{n}', f'Stop_Loss_{n}', f'SL_Type_{n}', f'CE_PNL_{n}', f'PE_PNL_{n}', f'Total_PNL_{n}'])                
            c = 0
            output_df.loc[c] = [ATM_strike, ep_ce, ep_pe, ep_straddle, entry_date, entry_weekday, entry_time, exit_date, exit_weekday, exit_time, exit_straddle , SL, sl_type, CE_PNL, PE_PNL, CE_PNL+PE_PNL]  
            data = pd.concat([df_merged, output_df], axis=1)
            datas = data.reindex(columns=sorted(list(data.columns)))
            mergefile.append(datas)
        else:
            output_df = pd.DataFrame(columns= ['Instrument', 'User_Entry_Time', 'User_Exit_Time', 'user_weekday','Stop_Loss_%', 'Max_Entries', 'Expiry', 'ATM_strike_1', 'Entry_Price_CE_1', 'Entry_Price_PE_1', 'Entry_Straddle_1','Entry_Date_1', 'Entry_Weekday_1', 'Entry_Time_1' , 'Exit_Date_1', 'Exit_Weekday_1','Exit_Time_1', 'Exit_Straddle_1', 'Stop_Loss_1', 'SL_Type_1' ,'CE_PNL_1', 'PE_PNL_1', 'Total_PNL_1'])
            c = 0
            output_df.loc[c] = [instrument, entry_time, exit_time, user_weekday, stop_loss_p, max_entries, expiry, ATM_strike, ep_ce, ep_pe, ep_straddle, entry_date, entry_weekday, entry_time, exit_date,exit_weekday, exit_time, exit_straddle , SL, sl_type, CE_PNL, PE_PNL, CE_PNL+PE_PNL]
            output_dfs = output_df.reindex(columns=sorted(list(output_df.columns)))
            mergefile.append(output_dfs)        
        
    df = pd.concat(mergefile, ignore_index=True)
    
    df.reset_index(drop=True, inplace=True)
    reindex_column = ['Instrument', 'Expiry', 'User_Entry_Time', 'User_Exit_Time', 'user_weekday','Stop_Loss_%', 'Max_Entries']
    
    for i in range(1, max(len_of_column) + 1):
        column_name_list_format = [f'Entry_Date_{i}', f'Entry_Weekday_{i}', f'Entry_Time_{i}', f'ATM_strike_{i}', f'Entry_Price_CE_{i}', f'Entry_Price_PE_{i}', f'Entry_Straddle_{i}', f'Stop_Loss_{i}', f'SL_Type_{i}', f'Exit_Date_{i}', f'Exit_Weekday_{i}', f'Exit_Time_{i}', f'Exit_Straddle_{i}', f'CE_PNL_{i}', f'PE_PNL_{i}', f'Total_PNL_{i}']
        reindex_column.extend(column_name_list_format)
    
    df = df.reindex(columns=reindex_column)    
    
    
    starting_time = (datetime.strptime(user_entry_time, "%H:%M:%S").time()).strftime("%H-%M-%S")
    exiting_time = (datetime.strptime(user_exit_time, "%H:%M:%S").time()).strftime("%H-%M-%S")

    folder_name = "Rolling_Straddle_Positional_Outputs"  
    file_starting_name = "PRolling_Straddle_Positional"  

    file_name = (f"{instrument}_{file_starting_name}_{starting_time}_{exiting_time}-{stop_loss_p}_{user_weekday}_{max_entries}.csv")

    output_file_path = (output_csv_path / folder_name /  f"{instrument}" / file_name)

    if not os.path.exists(output_csv_path / f"{folder_name}" / f"{instrument}"):
        os.makedirs(output_csv_path / f"{folder_name}" / f"{instrument}")

    if not os.path.exists(output_file_path):
        df.to_csv(output_file_path, mode='a', index=False, header=True)
    else: 
        df.to_csv(output_file_path, mode='a', index=False, header=False)
        read = pd.read_csv(output_file_path)
        df = read.drop_duplicates(subset='Expiry', keep="first")
        df.to_csv(output_file_path, index=False, header=True)
    print("Done 😊")

    finish = t.perf_counter()
    print(f'Finished in {round(finish-start, 2)} second(s)', "\n")

In [8]:
def Driver_Code(option_data, expiry_key_list, param_data):
    for params in param_data.itertuples():  
        entry_time = params.Entry_Time      
        exit_time = params.Exit_Time
        stop_loss_p = int(params.StopLoss)
        user_weekday = (params.Entry_Weekday).capitalize()
        max_entries = int(params.Max_Entries)  
        if ( entry_time > exit_time ) & (user_weekday == "Thursday"):
            print("{} Entry Time is greater than {} on Expiry Day {}, so entry not possible \n".format(entry_time, exit_time, user_weekday))
        else:
            Rolling_Straddle_Positional(option_data, expiry_key_list, entry_time, exit_time, stop_loss_p, user_weekday, max_entries)

In [9]:
%%time
filename, year = Filename(year_data, instrument)
root_path = data_root_path / instrument / "Option_Data" / filename
data_to_read = open(root_path, "rb")  
option_data = pickle.load(data_to_read)
expiry_key_list = sorted(list(set(list(option_data.keys())) & set(dates_range)))

Wall time: 1.23 s


In [10]:
%%time
Driver_Code(option_data, expiry_key_list, param_data)

Done 😊
Finished in 0.08 second(s) 

Done 😊
Finished in 0.08 second(s) 

Done 😊
Finished in 0.08 second(s) 

Done 😊
Finished in 0.09 second(s) 

Done 😊
Finished in 0.08 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.08 second(s) 

Done 😊
Finished in 0.08 second(s) 

Done 😊
Finished in 0.08 second(s) 

Done 😊
Finished in 0.09 second(s) 

Done 😊
Finished in 0.09 second(s) 

Done 😊
Finished in 0.08 second(s) 

Done 😊
Finished in 0.08 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.08 second(s) 

Done 😊
Finished in 0.18 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.07 seco

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.08 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.05 second(s) 

Done 😊
Finished in 0.05 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.05 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.07 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.06 seco

Done 😊
Finished in 0.06 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.05 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.05 second(s) 

Done 😊
Finished in 0.05 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.05 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 seco

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.04 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.03 seco

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.03 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.02 second(s) 

Done 😊
Finished in 0.01 second(s) 

Done 😊
Finished in 0.01 second(s) 

Done 😊
Finished in 0.01 second(s) 

Done 😊
Finished in 0.01 second(s) 

Done 😊
Finished in 0.01 second(s) 

Done 😊
Finished in 0.01 second(s) 

Done 😊
Finished in 0.01 second(s) 

Done 😊
Finished in 0.01 second(s) 

Done 😊
Finished in 0.01 second(s) 

Done 😊
Finished in 0.01 second(s) 

Done 😊
Finished in 0.01 second(s) 

15:25:59 Entry Time is greater than 15:20:59 on Expiry Day Thursday, so entry not possible 

Done 😊


In [11]:
dfx = pd.read_pickle(r"C:\Data\Dataset\Dict\NIFTY\Option_Data\NIFTY_2023.pkl")

In [13]:
dfx['2023-02-09'].keys()

dict_keys(['2023-02-03', '2023-02-06', '2023-02-07'])

In [15]:
dfx['2023-02-02'].keys()

dict_keys(['2023-01-27', '2023-01-30', '2023-01-31', '2023-02-01', '2023-02-02'])